In [2]:
%reload_ext autoreload
%autoreload 2

import os
from scipy.stats import truncnorm
import pickle

import corner
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline   

from models_streams import *
from models_orbits import *
from priors import *
from utils import *
from mock_data import *
from likelihoods import *
from dynesty_fit import dynesty_fit
from corner_plots import *

In [3]:
q_mean, q_sig, nlive, sigma, N, seed = 1.0, 0.1, 4000, 5, 100, 41
PATH_SAVE  = f'/data/dc824-2/final_orbit_to_stream/q{q_mean}_qsig{q_sig}_seed{seed}_nlive{nlive}_sigma{sigma}'
id = 4
save_stream = f'{PATH_SAVE}/xx_{id+1:03d}'

with open(f'{save_stream}/dict_data.pkl', 'rb') as file:
        dict_data = pickle.load(file)
params_data = np.loadtxt(f'{save_stream}/params.txt')
theta_initial = dict_data['theta'][0]

array([-1.23399 ,  0.923397,  0.254466])

In [4]:
xyz_stream, xyz_prog = gala_stream_model_ndim16(params_data)

In [17]:
import plotly.graph_objects as go

fig = go.Figure()

# Plot the stream
fig.add_trace(go.Scatter3d(
    x=xyz_stream[:, 0], 
    y=xyz_stream[:, 1], 
    z=xyz_stream[:, 2], 
    mode='markers', 
    marker=dict(size=5, color='blue'),
    name='Stream'
))

# Plot the final position of the progenitor on top
fig.add_trace(go.Scatter3d(
    x=[xyz_prog[0]], 
    y=[xyz_prog[1]], 
    z=[xyz_prog[2]], 
    mode='markers', 
    marker=dict(size=20, color='black'),
    name='Progenitor'
))

# Normalize the direction vector
d = params_data[3:6]
d_norm = np.linalg.norm(d)
if d_norm == 0:
    raise ValueError("Direction vector must be nonzero.")
d = d / d_norm

# Add a thick green line with orientation (dirx, diry, dirz)
fig.add_trace(go.Scatter3d(
    x=[-d[0] * 200, d[0] * 200], 
    y=[-d[1] * 200, d[1] * 200], 
    z=[-d[2] * 200, d[2] * 200], 
    mode='lines', 
    line=dict(color='green', width=10),
    showlegend=False
))

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    width=800,
    height=600
)

fig.show()